In [136]:
import os
import multiprocessing
import logging
from typing import Optional
import sys
import itertools

import geopandas as gpd
import numpy as np
import pandas as pd
import pyproj
import rioxarray
import xarray as xr
import matplotlib.pyplot as plt
import rasterio
import math

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [137]:
# Load netcdf with wind speeds
file = '/home/mark/projects/open-gira/results/direct/WP/max_wind_field_WP_1.nc'
ds = xr.open_dataset(file)
# Adjust the lat / lon coordinates to ensure they are counting cell corners (currently doing cell centres)
ds = ds.assign_coords({
    'longitude': ds.coords['longitude'] - 0.05,
    'latitude': ds.coords['latitude'] + 0.05
})
ds

<xarray.Dataset>
Dimensions:         (event_id: 100, latitude: 700, longitude: 850)
Coordinates:
  * event_id        (event_id) object 'WP_0_0_1' 'WP_0_0_14' ... 'WP_0_107_12'
  * latitude        (latitude) float64 65.0 64.9 64.8 64.7 ... -4.7 -4.8 -4.9
  * longitude       (longitude) float64 95.0 95.1 95.2 ... 179.7 179.8 179.9
Data variables:
    spatial_ref     int64 ...
    max_wind_speed  (event_id, latitude, longitude) float64 ...

In [138]:
# Add event_ids to list (so that we can iterate over the storms)
storms = ds.coords['event_id'].values.tolist()

In [139]:
# Load the static datasets
gdp_data = rasterio.open('/home/mark/projects/open-gira/results/direct/WP/GDP2005_0p1_WP.tif')
natID_data = rasterio.open('/home/mark/projects/open-gira/results/direct/WP/gpw4_natID_0p1_WP.tif')
gdp_raster = gdp_data.read(1)
natID_raster = natID_data.read(1)
tc_country_damage_lookup = pd.read_csv('/home/mark/projects/open-gira/results/direct/WP/TC_country_damage_look_up.csv')
damage_functions = pd.read_csv('/home/mark/projects/open-gira/results/direct/WP/Eberenz_damage_functions.csv')

In [140]:
tc_country_damage_lookup.head

<bound method NDFrame.head of               Name  ISO damage               Continent
0            Aruba  533    NA1  Americas-North America
1      Afghanistan    4     NI                    Asia
2           Angola   24      X                  Africa
3         Anguilla  660    NA1  Americas-North America
4    Åland Islands  248      X                  Europe
..             ...  ...    ...                     ...
236  Western Samoa  882     OC                 Oceania
237          Yemen  887     NI                    Asia
238   South Africa  710     SI                  Africa
239         Zambia  894      X                  Africa
240       Zimbabwe  716     SI                  Africa

[241 rows x 4 columns]>

In [141]:
damage_functions.head

<bound method NDFrame.head of   Label              Region  v_half_unc  v_half_rms  v_half_tdr  v_thresh
0   NA1                 NA1        74.7        59.6        66.3      25.7
1   NA2                 NA2        74.7        86.0        89.2      25.7
2    NI                  NI        74.7        58.7        70.8      25.7
3    OC                  OC        74.7        49.7        64.1      25.7
4    SI                  SI        74.7        46.8        52.4      25.7
5   WP1                 WP1        74.7        56.7        66.4      25.7
6   WP2                 WP2        74.7        84.7       188.4      25.7
7   WP3                 WP3        74.7        80.2       112.8      25.7
8   WP4                 WP4        74.7       135.6       190.5      25.7
9     X  Global Calibration        74.7        73.4       110.1      25.7>

In [142]:
# Define functions

# To speed up operations we want to look up in dictionaries rather than dataframes
# Convert DataFrame to dictionary for faster lookup
damage_dict = damage_functions.set_index('Label')['v_half_tdr'].to_dict() # Use TDR optimization as per Eberenz et. al (2021)
tc_country_damage_dict = tc_country_damage_lookup.set_index('ISO')['damage'].to_dict()

def damage_function(wind_speed, value, v_thresh, v_half):
    '''
    Given a wind_speed, GDP value, and damage function factors, returns a GDP damage value
    '''
    
    numerator = max([(wind_speed - v_thresh), 0])
    denominator = v_half - v_thresh
    v_n = numerator/denominator
    fractional_damage = math.pow(v_n, 3)/(1+math.pow(v_n, 3))
    damage = fractional_damage * value

    return damage

def apply_damage_function(gdp_value, wind_speed, national_ID):
    '''
    Lookup damage function factors based on national_ID - then apply damage function to all cells.
    '''
    basin = tc_country_damage_dict.get(str(national_ID))
    if basin:
        v_half = damage_dict.get(basin)
        if v_half is not None:
            damage = damage_function(wind_speed, gdp_value, 25.7, v_half)
    else:
        damage = None

    return damage

# Vectorize damage function
vectorized_damage_function = np.vectorize(apply_damage_function)

In [143]:
### Loop through storms applying damage function and saving results
# Info for saving results
country_ids = tc_country_damage_lookup['ISO'].unique()
country_names = tc_country_damage_lookup.set_index('ISO')['Name'].to_dict()
columns = ['Event ID', 'Total Damage'] + [country_names[id] for id in country_ids]
results_list = [] 
for i in storms:
    tc_event = ds['max_wind_speed'].sel(event_id=str(i)).to_numpy()
    tc_damage = vectorized_damage_function(gdp_raster, tc_event, natID_raster)

    # Create a dictionary to hold summary data for this storm
    summary_data = {'Event ID': i, 'Total Damage': np.nansum(tc_damage)}
    
    # Loop through each country
    for country_id in country_ids:
        # mask damage array to include only cells for this country and sum damage
        if country_id == '999 - not UN defined':
            country_mask = np.where(natID_raster == 999, 1, 0)
        else:
            country_mask = np.where(natID_raster == int(country_id), 1, 0)
        country_damage = np.nansum(tc_damage * country_mask)
        country_name = country_names[country_id]
        summary_data[country_name] = country_damage

    # Append to results list
    results_list.append(summary_data)

# Create a results dataframe
results_df = pd.DataFrame(results_list, columns=columns)

In [144]:
results_df.to_csv('/home/mark/projects/open-gira/results/direct/WP/wp_1_results.csv')